In [20]:
import os
import cv2
import numpy as np
import imutils
import random
import time
import sys
import csv

from PIL import ImageFont, ImageDraw, Image  

DATA_DIRECTORY = "C:/Users/Sander/Desktop/data/"

In [21]:
# Overlays a transparent image onto another transparent image
def insertTransparentImage(image, overlay, x1, x2, y1, y2):
    alpha_overlay = overlay[:, :, 3] / 255.0
    alpha_image = 1.0 - alpha_overlay

    # Intersect by alpha-value
    for c in range(0, 3):
        image[y1:y2, x1:x2, c] = (alpha_overlay * overlay[:, :, c] + alpha_image * image[y1:y2, x1:x2, c])
        
    return image

In [22]:
# Fits a text line by removing characters until the text width is smaller than "space"
def fitTextToSize(text, space, font, draw):
    res = ""
    
    # Add character-by-character and check if we are still within the valid width
    for c in text:
        res += c
        
        # Get new text width
        w, _ = draw.textsize(res, font=font)
        
        # Abort if new text width is too large
        if w > space:
            res = res[:len(res)-2]
            break
            
    return res

In [23]:
# Fills a notification with random text
def fillWithText(notification, padding, left_padding, top_padding, right_padding, bottom_padding, font_bold, font_bold_color, font_normal, font_normal_color):
    # Our vocabulary
    words = ["lorem", "ipsum", "dolor", "sit", "amet", "consetetur", "sadipscing", "elitr", "sed", 
             "diam", "nonumy", "eirmod", "tempor", "invidunt", "ut", "labore", "et", "dolore", "magna", 
             "aliquyam", "erat", "sed", "diam", "voluptua", "at", "vero", "eos", "et", "accusam", "et", 
             "justo", "duo", "dolores", "rebum"]
    
    # Convert the image to RGB (OpenCV uses BGR)
    notification = cv2.cvtColor(notification.astype('uint8'), cv2.COLOR_BGRA2RGBA)

    # Pass the image to PIL  
    notification_pil = Image.fromarray(notification)  
    draw = ImageDraw.Draw(notification_pil)  

    # use a truetype font  
    _, h = draw.textsize("h", font=font_normal)

    # Get allowed textwidth such that it fits in notification
    textWidth = notification.shape[1] - left_padding - right_padding
    
    # Get text paddings
    textX = left_padding
    textY = padding
    
    # Create random title
    text_title = " ".join(random.choices(words, k=random.randint(1,4)))
    text_title = fitTextToSize(text_title, textWidth, font_bold, draw)
    
    # Add random title
    draw.text((textX, textY), text_title, font=font_bold, fill=font_bold_color)
    
    # Add further text for each line below of the title
    while True:
        # Update vertical offset
        textY = textY + h + 5
        
        # Check if a new line still fits
        if textY + h + bottom_padding > notification.shape[0]:
            break
        
        # Check if we are below the notification image
        if textY >= top_padding:
            #textX = padding
            #textWidth = notification.shape[1] - padding - right_padding
            
            # Dont change anything
            pass

        # Create random textline
        text = " ".join(random.choices(words, k=random.randint(4,10)))
        text = fitTextToSize(text, textWidth, font_normal, draw)
        
        # Add random textline
        draw.text((textX, textY), text, font=font_normal, fill=font_normal_color)

    # Get back the image to OpenCV  
    notification = cv2.cvtColor(np.array(notification_pil), cv2.COLOR_RGBA2BGRA)
    
    return notification

In [24]:
# Creates a random windows desktop notification
def createWindowsNotification(size_x, size_y, icon_path, mode):
    # Background color of notification
    color = (31, 31, 31, 255)
    notification = np.ones((size_y, size_x, 4)) * color
    
    # Border color of notification
    border_color = (72, 72, 72, 255)
    
    # Add border to notification
    w = 2
    notification[0:w, 0:notification.shape[1]] = border_color
    notification[0:notification.shape[0], notification.shape[1]-w:notification.shape[1]] = border_color
    notification[notification.shape[0]-w:notification.shape[0], 0:notification.shape[1]] = border_color
    notification[0:notification.shape[0], 0:w] = border_color
    
    # Text and image padding of notification
    padding = 12
    
    # Padding for the text part (left)
    left_padding = padding
    
    # Padding for the text part (right)
    right_padding = padding
    
    # Height of where the icon is finished, in case we also want to shift the text below the icon
    top_padding = padding
    
    # Padding of the text part (bottom)
    bottom_padding = padding
    
    # If an icon was provided, add it to the left side of the notification
    if icon_path is not None:
        # Read image
        icon = cv2.imread(icon_path, cv2.IMREAD_UNCHANGED)

        # Resize to correct size
        icon = imutils.resize(icon, height=min(int(size_y / 2), icon.shape[0]))
        icon = imutils.resize(icon, width=min(int(size_x / 4), icon.shape[1]))
        
        # Get position of icon within notification
        x1 = min(padding, int((notification.shape[1] / 2) - (icon.shape[1] / 2)))
        x2 = x1 + icon.shape[1]
        
        y1 = min(padding, int((notification.shape[0] / 2) - (icon.shape[0] / 2)))
        y2 = y1 + icon.shape[0]
    
        # Add notification on image
        notification = insertTransparentImage(notification, icon, x1, x2, y1, y2)
        
        # Update the text paddings accordingly
        left_padding = left_padding + x2
        top_padding = top_padding + y2
    
    # Get fonts
    path_to_additional_data = DATA_DIRECTORY + "/windows_files/data/"
    path_to_font_bold = os.path.join(path_to_additional_data, 'SegoeUI_Bold.ttf')
    path_to_font_normal = os.path.join(path_to_additional_data, 'SegoeUI.ttf')
    
    fontsize = 14
    font_bold = ImageFont.truetype(path_to_font_bold, fontsize)
    font_normal = ImageFont.truetype(path_to_font_normal, fontsize)
    
    # In case notification is large enough, and "mode" allows it, add action buttons to right side of notification
    if size_x >= 250 and size_y >= 120 and mode > 0:
        # Height of action button
        button_height = 30
        
        # Update bottom text padding
        bottom_padding = padding + button_height + padding
        
        # x-coordinate of middle
        mid = int(notification.shape[1] / 2)
        
        # Color of action button
        button_color = (80, 73, 67, 255)
        
        # Draw correct button background
        if mode == 1:
            x1 = padding
            x2 = notification.shape[1] - padding
            
            y1 = notification.shape[0] - bottom_padding + padding
            y2 = y1 + button_height
            
            notification[y1:y2, x1:x2] = button_color
        elif mode == 2:
            x1 = padding
            x2 = mid - int(padding/2)
            
            y1 = notification.shape[0] - bottom_padding + padding
            y2 = y1 + button_height
            
            notification[y1:y2, x1:x2] = button_color
            
            x1 = mid + int(padding/2)
            x2 = notification.shape[1] - padding
            
            notification[y1:y2, x1:x2] = button_color
            
        # Our action vocabulary
        words = ["Lorem", "Ipsum", "Dolor", "Consetetur", "Sadipscing", "Elitr", "Nonumy", "Eirmod", "Tempor", "Invidunt",
                 "Labore", "Dolore", "Magna", "Aliquyam", "Erat", "Voluptua", "Vero", "Accusam", "Dolores", "Rebum"]
        
        # Convert the image to RGB (OpenCV uses BGR)
        notification = cv2.cvtColor(notification.astype('uint8'), cv2.COLOR_BGRA2RGBA)

        # Pass the image to PIL  
        notification_pil = Image.fromarray(notification)  
        draw = ImageDraw.Draw(notification_pil)  

        # Start Y-coordinate of button
        button_start = notification.shape[0] - bottom_padding + padding
        
        # Check if there is only one action button (mode 1)
        if mode == 1:
            # Get random word
            text = random.choice(words)
            w, _ = draw.textsize(text, font=font_normal)
            _, h = draw.textsize("h", font=font_normal)
            
            # Get text coordinates
            textX = int((notification.shape[1] - w) / 2)
            textY = button_start + int((button_height - h) / 2) - 1
            
            # Draw random word
            draw.text((textX, textY), text, font=font_normal, fill=(255, 255, 255, 255))
        elif mode == 2:
            # We want two action buttons (mode 2)
            # Get two random words
            text = random.sample(words, k=2)
            _, h = draw.textsize("h", font=font_normal)
            
            # Get text coordinates (upper word)
            w, _ = draw.textsize(text[0], font=font_normal)
            textX = (mid - int(padding/2) - padding - w) / 2 + padding
            textY = button_start + int((button_height - h) / 2) - 1
            
            # Draw upper word
            draw.text((textX, textY), text[0], font=font_normal, fill=(255, 255, 255, 255))

            # Get text coordinates (lower word)
            w, _ = draw.textsize(text[1], font=font_normal)
            textX = (mid - int(padding/2) - padding - w) / 2 + mid + int(padding/2)

            # Draw lower word
            draw.text((textX, textY), text[1], font=font_normal, fill=(255, 255, 255, 255))
            
        # Get back the image to OpenCV  
        notification = cv2.cvtColor(np.array(notification_pil), cv2.COLOR_RGBA2BGRA)
        
    # Get font colors
    font_bold_color = (255, 255, 255, 255)
    font_normal_color = (165, 165, 165, 255)
    
    # Fill notification with text
    notification = fillWithText(notification, padding, left_padding, top_padding, right_padding, bottom_padding, font_bold, font_bold_color, font_normal, font_normal_color)
    
    return notification

In [25]:
# Creates a random mac desktop notification
def createMacNotification(size_x, size_y, icon_path, mode):
    # Background color of notification
    color = (240, 240, 240, 255)
    notification = np.ones((size_y, size_x, 4)) * color
    
    # Load the "round" corner of the notification
    path_to_additional_data = DATA_DIRECTORY + "/mac_files/data/"
    corner = cv2.imread(os.path.join(path_to_additional_data, "notification_corner.png"), cv2.IMREAD_UNCHANGED)
    
    # Border color of notification (transparent effect)
    border_color = (0, 0, 0, 35)
    
    # Add border effect (outer ring)
    notification[0:1, 0:notification.shape[1]] = border_color
    notification[0:notification.shape[0], notification.shape[1]-1:notification.shape[1]] = border_color
    notification[notification.shape[0]-1:notification.shape[0], 0:notification.shape[1]] = border_color
    notification[0:notification.shape[0], 0:1] = border_color
    
    # Border color of notification (a little bit less transparent)
    border_color = (0, 0, 0, 46)
    
    # Add border effect (inner ring)
    notification[1:2, 1:notification.shape[1]] = border_color
    notification[1:notification.shape[0]-1, notification.shape[1]-2:notification.shape[1]-1] = border_color
    notification[notification.shape[0]-2:notification.shape[0]-1, 1:notification.shape[1]-1] = border_color
    notification[1:notification.shape[0]-1, 1:2] = border_color
    
    # Add the "round" corners by rotating the image accrodingly
    notification[0:corner.shape[0], 0:corner.shape[1]] = corner
    notification[0:corner.shape[0], notification.shape[1]-corner.shape[1]:notification.shape[1]] = imutils.rotate(corner, angle=270)
    notification[notification.shape[0]-corner.shape[0]:notification.shape[0], 0:corner.shape[1]] = imutils.rotate(corner, angle=90)
    notification[notification.shape[0]-corner.shape[0]:notification.shape[0], notification.shape[1]-corner.shape[1]:notification.shape[1]] = imutils.rotate(corner, angle=180)
    
    # Text and image padding of notification
    padding = 12 + 2
    
    # Padding for the text part (left)
    left_padding = padding
    
    # Padding for the text part (right)
    right_padding = padding
    
    # Height of where the icon is finished, in case we also want to shift the text below the icon
    top_padding = padding
    
    # Padding of the text part (bottom)
    bottom_padding = padding
    
    # If an icon was provided, add it to the left side of the notification
    if icon_path is not None:
        # Read image
        icon = cv2.imread(icon_path, cv2.IMREAD_UNCHANGED)

        # Resize to correct size
        icon = imutils.resize(icon, height=min(int(size_y / 2), icon.shape[0]))
        icon = imutils.resize(icon, width=min(int(size_x / 4), icon.shape[1]))
        
        # Get position of icon within notification
        x1 = min(padding, int((notification.shape[1] / 2) - (icon.shape[1] / 2)))
        x2 = x1 + icon.shape[1]
        
        y1 = int((notification.shape[0] / 2) - (icon.shape[0] / 2))
        y2 = y1 + icon.shape[0]
    
        # Add notification on image
        notification = insertTransparentImage(notification, icon, x1, x2, y1, y2)
        
        # Update the text paddings accordingly
        left_padding = left_padding + x2
    
    # Get fonts
    path_to_font_bold = os.path.join(path_to_additional_data, 'LucidaGrandeBold.ttf')
    path_to_font_normal = os.path.join(path_to_additional_data, 'LucidaGrande.ttf')
    
    fontsize = 14
    font_bold = ImageFont.truetype(path_to_font_bold, fontsize)
    font_normal = ImageFont.truetype(path_to_font_normal, fontsize)
    
    # In case notification is large enough, and "mode" allows it, add action buttons to right side of notification
    if size_x >= 250 and mode > 0:
        # Width of action buttons
        width = 80
        
        # Update right text padding
        right_padding = padding + width
        
        # x-coordinate of vertical line
        start = notification.shape[1] - right_padding
        
        # y-coordinate of horizontal line
        mid = int(notification.shape[0] / 2)
        
        # Our action vocabulary
        words = ["Lorem", "Ipsum", "Dolor", "Consetetur", "Sadipscing", "Elitr", "Nonumy", "Eirmod", "Tempor", "Invidunt",
                 "Labore", "Dolore", "Magna", "Aliquyam", "Erat", "Voluptua", "Vero", "Accusam", "Dolores", "Rebum"]
            
        # Add vertical line
        notification[2:notification.shape[0]-2, start:start+2] = (217, 217, 217, 255)
        
        # If we want two action buttons (mode 2), add horizontal line too
        if mode == 2:
            notification[mid-1:mid+1, start:notification.shape[1]-2] = (217, 217, 217, 255)
        
        # Convert the image to RGB (OpenCV uses BGR)
        notification = cv2.cvtColor(notification.astype('uint8'), cv2.COLOR_BGRA2RGBA)

        # Pass the image to PIL  
        notification_pil = Image.fromarray(notification)  
        draw = ImageDraw.Draw(notification_pil)  
        
        # Check if there is only one action button (mode 1)
        if mode == 1:
            # Get random word
            text = random.choice(words)
            w, h = draw.textsize(text, font=font_normal)

            # Get text coordinates
            textX = notification.shape[1] - right_padding + int((right_padding - w) / 2)
            textY = mid - int(h / 2)
            
            # Draw random word
            draw.text((textX, textY), text, font=font_normal, fill=(113, 113, 113, 255))
        elif mode == 2:
            # We want two action buttons (mode 2)
            # Get two random words
            text = random.sample(words, k=2)
            
            # Get text coordinates (upper word)
            w, h = draw.textsize(text[0], font=font_normal)
            textX = notification.shape[1] - right_padding + int((right_padding - w) / 2)
            textY = int(mid / 2) - int(h / 2)
            
            # Draw upper word
            draw.text((textX, textY), text[0], font=font_normal, fill=(113, 113, 113, 255))

            # Get text coordinates (lower word)
            w, h = draw.textsize(text[1], font=font_normal)
            textX = notification.shape[1] - right_padding + int((right_padding - w) / 2)
            textY = mid + int(mid / 2) - int(h / 2)
            
            # Draw lower word
            draw.text((textX, textY), text[1], font=font_normal, fill=(113, 113, 113, 255))
            
        # Get back the image to OpenCV  
        notification = cv2.cvtColor(np.array(notification_pil), cv2.COLOR_RGBA2BGRA)
    
    # Get font colors
    font_bold_color = (77, 77, 77, 255)
    font_normal_color = (77, 77, 77, 255)
    
    # Fill notification with text
    notification = fillWithText(notification, padding, left_padding, top_padding, right_padding, bottom_padding, font_bold, font_bold_color, font_normal, font_normal_color)
    
    return notification

In [26]:
# Creates a random linux desktop notification
def createLinuxNotification(size_x, size_y, icon_path):
    # Background color of notification
    color = (48, 51, 52, 255)
    notification = np.ones((size_y, size_x, 4)) * color
    
    # Load the "round" corner of the notification
    path_to_additional_data = DATA_DIRECTORY + "/linux_files/data/"
    corner = cv2.imread(os.path.join(path_to_additional_data, "notification_corner.png"), cv2.IMREAD_UNCHANGED)
    
    # Border color of notification (transparent effect)
    border_color = (0, 0, 0, 80)
    
    # Add border effect (outer ring)
    notification[0:1, 0:notification.shape[1]] = border_color
    notification[0:notification.shape[0], notification.shape[1]-1:notification.shape[1]] = border_color
    notification[notification.shape[0]-1:notification.shape[0], 0:notification.shape[1]] = border_color
    notification[0:notification.shape[0], 0:1] = border_color
    
    # Border color of notification (a little bit less transparent)
    border_color = (45, 47, 48, 198)
    
    # Add border effect (inner ring)
    notification[1:2, 1:notification.shape[1]] = border_color
    notification[1:notification.shape[0]-1, notification.shape[1]-2:notification.shape[1]-1] = border_color
    notification[notification.shape[0]-2:notification.shape[0]-1, 1:notification.shape[1]-1] = border_color
    notification[1:notification.shape[0]-1, 1:2] = border_color
    
    # Add the "round" corners by rotating the image accrodingly
    notification[0:corner.shape[0], 0:corner.shape[1]] = corner
    notification[0:corner.shape[0], notification.shape[1]-corner.shape[1]:notification.shape[1]] = imutils.rotate(corner, angle=270)
    notification[notification.shape[0]-corner.shape[0]:notification.shape[0], 0:corner.shape[1]] = imutils.rotate(corner, angle=90)
    notification[notification.shape[0]-corner.shape[0]:notification.shape[0], notification.shape[1]-corner.shape[1]:notification.shape[1]] = imutils.rotate(corner, angle=180)
    
    # Text and image padding of notification
    padding = 12 + 2
    
    # Padding for the text part (left)
    left_padding = padding
    
    # Padding for the text part (right)
    right_padding = padding
    
    # Height of where the icon is finished, in case we also want to shift the text below the icon
    top_padding = padding
    
    # Padding of the text part (bottom)
    bottom_padding = padding
    
    # If an icon was provided, add it to the left side of the notification
    if icon_path is not None:
        # Read image
        icon = cv2.imread(icon_path, cv2.IMREAD_UNCHANGED)

        # Resize to correct size
        icon = imutils.resize(icon, height=min(int(size_y / 2), icon.shape[0]))
        icon = imutils.resize(icon, width=min(int(size_x / 4), icon.shape[1]))
        
        # Get position of icon within notification
        x1 = min(padding, int((notification.shape[1] / 2) - (icon.shape[1] / 2)))
        x2 = x1 + icon.shape[1]
        
        y1 = min(padding, int((notification.shape[0] / 2) - (icon.shape[0] / 2)))
        y2 = y1 + icon.shape[0]
    
        # Add notification on image
        notification = insertTransparentImage(notification, icon, x1, x2, y1, y2)
        
        # Update the text paddings accordingly
        left_padding = left_padding + x2
        top_padding = top_padding + y2

    # Get fonts
    path_to_font_bold = os.path.join(path_to_additional_data, 'Ubuntu-Medium.ttf')
    path_to_font_normal = os.path.join(path_to_additional_data, 'Ubuntu-Light.ttf')
    
    fontsize = 14
    font_bold = ImageFont.truetype(path_to_font_bold, fontsize)
    font_normal = ImageFont.truetype(path_to_font_normal, fontsize)
    
    # Get font color
    font_bold_color = (255, 255, 255, 255)
    font_normal_color = (255, 255, 255, 255)
    
    # Fill notification with text
    notification = fillWithText(notification, padding, left_padding, top_padding, right_padding, bottom_padding, font_bold, font_bold_color, font_normal, font_normal_color)
    
    return notification

In [27]:
# Add a notification randomly to a desktop screenshot
def addNotification(desktop, design_layout):
    # Min and max possible width of a notification
    min_width = int(desktop.shape[1] / 20)
    max_width = min_width * 4
    
    # Min and max possible height of a notification
    min_height = int(desktop.shape[0] / 16)
    max_height = min_height * 3
    
    # Choose a random width and height
    size_x = random.randint(min_width, max_width)
    size_y = random.randint(min_height, max_height)
    
    # Choose a random position (upper left corner of notification)
    x = random.randint(0, desktop.shape[1] - size_x)
    y = random.randint(0, desktop.shape[0] - size_y)

    # Randomly decide if we want to add a icon
    coinflip = random.randint(0, 4)
    icon_path = None

    # Load a random icon (if coinflip number is larger than 0)
    if size_x >= 120 and coinflip > 0:
        icon_directory = DATA_DIRECTORY + "/icons/"
        icon_name = random.choice(os.listdir(icon_directory))
        icon_path = os.path.join(icon_directory, icon_name)
    
    # No action buttons for mac notification
    mac_mode = 0
    
    if size_x >= 250:
        # Randomly decide if we want to add action buttons (to a mac notification)
        mac_mode = random.randint(0, 2)
        
    # No action buttons for windows notification
    windows_mode = 0
    
    if size_x >= 250 and size_y >= 120:
        # Randomly decide if we want to add action buttons (to a windows notification)
        windows_mode = random.randint(0, 2)
    
    # Create the correct design layout
    if design_layout == "Windows":
        notification = createWindowsNotification(size_x, size_y, icon_path, windows_mode)
    elif design_layout == "Mac":
        notification = createMacNotification(size_x, size_y, icon_path, mac_mode)
    elif design_layout == "Linux":
        notification = createLinuxNotification(size_x, size_y, icon_path)
    else:
        print("Unknown design layout:", design_layout)
    
    # Provide multiple ranges we can pick our opacity from (for fixed location and size)
    intervals = [[0.8, 1.0]]
    #intervals = [[0.15, 0.35], [0.35, 0.5], [0.5, 0.7], [0.7, 0.85], [0.85, 1.0]]
    
    # Batch holding all created screenhost
    images =[]
    values = []
    
    # Pick a random opacity for each provided range
    for inv in intervals:
        # Load desktop screenshot
        image = cv2.cvtColor(desktop, cv2.COLOR_RGB2RGBA).copy()
        
        # Pick opacity
        opacity = int(random.uniform(inv[0], inv[1]) * 255)

        # Apply opacity to notification
        new_notification = notification.copy()
        new_notification[:, :, 3] = notification[:, :, 3] * (opacity / 255)
        
        # Add notifiction to screenshot
        image = insertTransparentImage(image, new_notification, x, x+size_x, y, y+size_y)
    
        # Append to batch
        images.append(image)
        values.append([x, y, size_x, size_y, opacity])
    
    # Provide multiple sizes (for fixed location and opacity)
    intervals = [[size_x, size_y]]
    #intervals = [[min_width, min_height], [min_width, max_height], [max_width, min_height], [min_width * 3, int(min_height * 1.5)], [max_width, max_height]]
    
    # Create a notification for each provided size
    for inv in intervals:
        # Load desktop screenshot
        image = cv2.cvtColor(desktop, cv2.COLOR_RGB2RGBA).copy()
        
        # Opacity is fixed
        opacity = 255

        # Pick size
        size_x = inv[0]
        size_y = inv[1]
        
        # Maybe move notification in case the new size exceeds the screenshot width/height
        new_x = min(x, desktop.shape[1] - size_x)
        new_y = min(y, desktop.shape[0] - size_y)
        
        # Create the correct notification
        if design_layout == "Windows":
            notification = createWindowsNotification(size_x, size_y, icon_path, windows_mode)
        elif design_layout == "Mac":
            notification = createMacNotification(size_x, size_y, icon_path, mac_mode)
        elif design_layout == "Linux":
            notification = createLinuxNotification(size_x, size_y, icon_path)
        else:
            print("Unknown design layout:", design_layout)
        
        # Apply opacity to notification
        notification[:, :, 3] = notification[:, :, 3] * (opacity / 255)
        
        # Add notifiction to screenshot
        image = insertTransparentImage(image, notification, new_x, new_x+size_x, new_y, new_y+size_y)
    
        # Append to batch
        images.append(image)
        values.append([new_x, new_y, size_x, size_y, opacity])
    
    return images, values

In [28]:
# Provide test data
design_layout = "Windows"
test_path = "C:/Users/Sander/Desktop/Dataset/Original/" + design_layout + "/"

# Pick random screenshot from test data
files = os.listdir(test_path)
file = random.choice(files)

# Add notifcation to test screenshot
image = cv2.imread(os.path.join(test_path, file))
images, values = addNotification(image, design_layout)

# Display test image within screenshot
idx = len(images)-1-1
print("Opacity:", values[idx][4])
print("Size:", values[idx][2], "x", values[idx][3])
print("Location:", values[idx][0], "/", values[idx][1])

cv2.imshow("Desktop with Notification", images[idx])
cv2.waitKey(0)
cv2.destroyAllWindows()

FileNotFoundError: [WinError 3] Das System kann den angegebenen Pfad nicht finden: 'C:/Users/Sander/Desktop/Dataset/Original/Windows/'

In [29]:
# Display a test notification without the screenshot
icon_directory = DATA_DIRECTORY + "icons/"

# Select random icon
icon_name = random.choice(os.listdir(icon_directory))
icon_path = os.path.join(icon_directory, icon_name)

# Get max and min possible width/height
min_width = int(1920 / 20)
max_width = min_width * 4
    
min_height = int(1080 / 16)
max_height = min_height * 3

# Decide for width/height
size_x = max_width
size_y = max_height
    
# Create notification
#notification = createWindowsNotification(size_x, size_y, icon_path)
notification = createMacNotification(size_x, size_y, icon_path, 0)
#notification = createLinuxNotification(size_x, size_y, icon_path)

# Display and store notification
cv2.imshow("Notification", notification)
cv2.waitKey(0)
cv2.destroyAllWindows()

#cv2.imwrite("C:/Users/Sander/Desktop/" + str(int(time.time()))+ ".png", notification)

In [11]:
# Prints the progress in the console
def print_progress(count, total):
    pct_complete = float(count) / total
    msg = "\r- Progress: {0:.1%}".format(pct_complete)

    # Print it.
    sys.stdout.write(msg)
    sys.stdout.flush()

In [22]:
# Adds random notifications to the provided dataset with the above given batches in "addNotification()"

# Privide OS
design_layout = "Linux"

# Provide screenshot dataset
input_path = "C:/Users/Sander/Desktop/Dataset/Original/" + design_layout + "/"

# Provide output path for screenshot+notification images
output_path = "C:/Users/Sander/Desktop/Dataset/Notifications/" + design_layout + "/"

# Create csv file which holds notification values for each screenshot
csv_file = os.path.join(output_path, "data.out")
      
# Create missing paths
if not os.path.exists(output_path):
      os.mkdir(output_path)
        
if os.path.exists(csv_file):
      os.remove(csv_file)

print(input_path)
allfiles = sorted(os.listdir(input_path))

# Create notitifcations
with open(csv_file, "w", newline="") as f:
    writer = csv.writer(f)

    for i, filename in enumerate(allfiles):
        print_progress(i+1, len(allfiles))

        image = cv2.imread(os.path.join(input_path, filename))
        images, values = addNotification(image, design_layout)
        
        # For each notitifation in batch, store a separate image
        for i, img in enumerate(images):
            new_filename = filename.split(".")[0] + "_" + str(i+1) + ".png"
            cv2.imwrite(os.path.join(output_path, new_filename), img)
            writer.writerow([new_filename] + values[i])

print()
print("Terminated")

C:/Users/Sander/Desktop/Dataset/Original/Linux/
- Progress: 100.0%
Terminated


In [31]:
# Synthesize random notifications without putting them on screenshots

# Privide OS
design_layout = "Linux"

# Output path of synthesized notifications
output_path = "C:/Users/Sander/Desktop/Dataset/Out2/" + design_layout + "/"

if not os.path.exists(output_path):
    os.mkdir(output_path)

# Get max and min possible width/height
'''
min_width = int(1920 / 20)
max_width = min_width * 4
    
min_height = int(1080 / 16)
max_height = min_height * 3
'''

sizes = [[300, 80], [200, 150], [160, 250], [140, 350], [90, 385], [65, 420]]

# Define opacity
opacity = 255

# Number of synthesized screenshots
number_of_images = 100

# Create random screenshots
for i in range(0, number_of_images):
    print_progress(i+1, number_of_images)
    
    # Choose a random width and height
    idx = random.randint(0, len(sizes)-1)
    size_x = sizes[idx][1]
    size_y = sizes[idx][0]
    
    #size_x = random.randint(min_width, max_width)
    #size_y = random.randint(min_height, max_height)
    
    # Randomly decide if we want to add a icon
    coinflip = random.randint(0, 4)
    icon_path = None

    # Load a random icon (if coinflip number is larger than 0)
    if size_x >= 120 and coinflip > 0:
        icon_directory = DATA_DIRECTORY + "icons/"
        icon_name = random.choice(os.listdir(icon_directory))
        icon_path = os.path.join(icon_directory, icon_name)
    
    # No action buttons for mac notification
    mac_mode = 0
    
    if size_x >= 250:
        # Randomly decide if we want to add action buttons (to a mac notification)
        mac_mode = random.randint(0, 2)
        
    # No action buttons for mac notification
    windows_mode = 0
    
    if size_x >= 250 and size_y >= 120:
        # Randomly decide if we want to add action buttons (to a mac notification)
        windows_mode = random.randint(0, 2)
        
    # Create the correct notification
    if design_layout == "Windows":
        notification = createWindowsNotification(size_x, size_y, icon_path, windows_mode)
    elif design_layout == "Mac":
        notification = createMacNotification(size_x, size_y, icon_path, mac_mode)
    elif design_layout == "Linux":
        notification = createLinuxNotification(size_x, size_y, icon_path)
    else:
        print("Unknown design layout:", design_layout)
    
    # Apply opacity to notification
    notification[:, :, 3] = notification[:, :, 3] * (opacity / 255)
        
    cv2.imwrite(output_path + str(i+1) + ".png", notification)

print()
print("Terminated")

FileNotFoundError: [WinError 3] Das System kann den angegebenen Pfad nicht finden: 'C:/Users/Sander/Desktop/Dataset/Out2/Linux/'